**Note:** most of the code is extracted from [source](https://cameo.bio/05-predict-gene-knockout-strategies.html#optknock).

OptGene is an algorithm that tries to optimize production of a target metabolite an minimize production of undesired by-products by simulating the addition of metabolic pathways (Patil 2005).

# **OptGene**

In [2]:
# This is just importing of the model and some tools to work with the GSM.
import numpy as np
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
import pandas as pd
# Read model (central metabolism model of Escherichia coli)
model = read_sbml_model("iML1515.xml")
%run -i scripts/create_pathway.ipynb 

In [3]:
# creating the models with the respective pathways
nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [(nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

Read LP format model from file /var/folders/xm/9p7p9cbx5pl_147c9t396fx40000gp/T/tmpkpwgyzl3.lp
Reading time = 0.01 seconds
: 1883 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /var/folders/xm/9p7p9cbx5pl_147c9t396fx40000gp/T/tmpg2sxfimu.lp
Reading time = 0.02 seconds
: 1883 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /var/folders/xm/9p7p9cbx5pl_147c9t396fx40000gp/T/tmpt571pbru.lp
Reading time = 0.02 seconds
: 1883 rows, 5424 columns, 21150 nonzeros


In [ ]:
# creating the models with the respective pathways

# path_selector legend
# 0 -> no additional reactions
#1 -> nicotinamid pathway  
#2 -> nicotinic acid pathway  
#3 -> nicotinamid riboside pathway

nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [model, (nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]
'done'

In [ ]:

#Choosing uptake rates
medium=model.medium
medium['EX_ncam_e'] = 100
medium['EX_nr_e'] = 100
medium["EX_na_e"] = 100
medium["EX_glc__D_e"] = 10

In [ ]:
# Visualize the initial phenotypic phase plane
init_solution = model.optimize()
growth = init_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
NMN_production = init_solution.fluxes['NAMPT']
p = phenotypic_phase_plane(model, variables=['BIOMASS_Ec_iML1515_core_75p37M'], objective=['NAMPT'])
p.plot(plotter, points=[(growth, NMN_production)])